In [1]:
import arviz as az
import pandas as pd
import polars as pl
import os
import bambi as bmb

In [2]:
SIPP_MODEL_DATA = os.environ.get("SIPP_MODEL_DATA")
SIPP_MODEL_DATA

'D:\\new-orleans-local-wealth-profile_data\\sipp_model.csv'

In [ ]:
sipp_model = pl.read_parquet()
sipp_model

In [ ]:
model_cols = ['prank_assets',
'state',
'hh_income',
'age',
'race_eth',
'edu',
'tenure',
'household_type',
'male',
'metro',
'disability',
'class_worker',
'public_assistance',
'social_security',
'poverty',
'citizen',
'english_at_home',
'homevalue',]

In [ ]:
sipp_model['age'].value_counts()

In [ ]:
sipp_model['hh_income'].value_counts()

In [ ]:
df = sipp_model.select(pl.col(model_cols)).to_pandas().dropna(subset=model_cols)
df["race_eth_state"] = df["race_eth"].astype(str) + ":" + df["state"].astype(str)
df["race_eth_edu"] = df["race_eth"].astype(str) + ":" + df["edu"].astype(str)
df["race_eth_age"] = df["race_eth"].astype(str) + ":" + df["age"].astype(str)
df["race_eth_income"] = df["race_eth"].astype(str) + ":" + df["hh_income"].astype(str)
df

In [ ]:
cols = ["race_eth_state","race_eth_edu","race_eth_age","race_eth_income"]
{c: df[c].nunique(dropna=False) for c in cols}

In [ ]:
for c in cols:
    vc = df[c].value_counts()
    print(c, "levels:", vc.shape[0], "min:", vc.min(), "p1%:", vc.quantile(0.01), "median:", vc.median())

In [ ]:
df.isnull().sum()

In [ ]:
formula = """
prank_assets ~ male + metro + disability + class_worker + public_assistance
            + social_security + poverty + citizen + english_at_home + homevalue
            + (1|state) + (1|hh_income) + (1|age) + (1|race_eth) + (1|edu)
            + (1|tenure) + (1|household_type)
            + (1|race_eth_state) + (1|race_eth_edu) + (1|race_eth_age) + (1|race_eth_income)
"""

In [ ]:
# formula = """
# prank_assets ~ male + metro + disability + class_worker + public_assistance
#             + social_security + poverty + citizen + english_at_home + homevalue
#             + state hh_income + age + race_eth + edu
#             + tenure + household_type + (1|race_eth_edu) + (1|race_eth_age) + (1|race_eth_income)
# """

In [ ]:

priors = {
    "Intercept": bmb.Prior("Normal", mu=0, sigma=1),
    "Common": bmb.Prior("Normal", mu=0, sigma=1),  # fixed effects
    "GroupSpecific": bmb.Prior("HalfNormal", sigma=0.5),  # group SDs
    "1|race_eth_state": bmb.Prior("HalfNormal", sigma=0.25),
}

In [ ]:
group_specific_prior = bmb.Prior(
    "Normal",
    mu=0,
    sigma=bmb.Prior("HalfNormal", sigma=0.5)
)

In [ ]:
priors = {
    "Intercept": bmb.Prior("Normal", mu=0, sigma=1),
    "common": bmb.Prior("Normal", mu=0, sigma=1),

    # random effects coefficients ~ Normal(0, sigma_group), with sigma_group having its own prior
    "group_specific": bmb.Prior(
        "Normal",
        mu=0,
        sigma=bmb.Prior("HalfNormal", sigma=0.5),
    ),

    # residual SD (often helps stability)
    "sigma": bmb.Prior("HalfNormal", sigma=1),

    # optional: extra shrinkage for the sparse interaction's SD
    # only works if this name matches exactly what `print(model)` shows
    "1|race_eth_state_sigma": bmb.Prior("HalfNormal", sigma=0.25),
}

In [ ]:
model = bmb.Model(formula, df, family="gaussian",priors=priors)

In [ ]:
idata = model.fit(draws=4000, 
                  tune=4000, 
                  chains=4, 
                  target_accept=0.99,
                  inference_method="numpyro",
                  max_treedepth = 18)

In [ ]:
import arviz as az
az.to_netcdf(idata,r'D:\new-orleans-local-wealth-profile_data\model_results\asset_rankmod_v3_idata.nc')

In [ ]:
az.plot_pair(
    idata,
    var_names=["male", "metro"],  # add suspicious vars
    divergences=True,
    kind="kde",
)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_trace(idata)

In [ ]:
az.plot_posterior(idata, var_names=["male"])

In [ ]:
posterior_df = idata.posterior.to_dataframe().reset_index()
posterior_df


In [ ]:
idata